# Installation
Install from GitHub as described in [the docs](http://qcodes.github.io/Qcodes/start/index.html).

In [ ]:
import numpy as np
import qcodes as qc

# Instrument setup

At this point, each measurement setup needs to define which instruments they use. See [the docs](http://qcodes.github.io/Qcodes/user/intro.html#instrument). For demonstration purposes, here I just use dummy instruments `dac` and `dmm`.

In [ ]:
from qcodes.tests.instrument_mocks import DummyInstrument
dac = DummyInstrument(name="dac", gates=['ch1', 'ch2'])
dmm = DummyInstrument(name="dmm", gates=['idc', 'ig'])

import random
dmm.idc.get = lambda: random.gauss(1, 0.1)
dmm.ig.get = lambda: random.gauss(0, 0.01)

You can set `step` and `inter_delay` on any parameters. These will apply to all future attempts to set the value, including those in loops. For instance, if `dac.ch1` can be ramped at a maximum speed of 0.1 V / ms, then do this:

In [ ]:
dac.ch1.step = 0.1
dac.ch1.inter_delay = 0.001

# Experiment setup

The first time you run qcodes on a new computer, you need to create the SQLite database. See [the docs](https://qcodes.github.io/Qcodes/dataset/index.html) for details on the design of the dataset and database. You will then want to make a new experiment to hold your data. At the very least, make a new experiment when you change out your samples.

In [ ]:
qc.initialise_database()
qc.new_experiment(name='demo', sample_name='my best sample')

# Measurement

In [ ]:
import sweep

In [ ]:
s = sweep.Sweep()
s.follow_param(dmm.ig, gain=10)
s.follow_param(dmm.idc)

s.plot('time', dac.ch1)

# Sweep from 0 to 25.
s.sweep(dac.ch1, np.linspace(0, 25, 10), inter_delay=1)
# Sweep from 0 to 25 then back to 0.
s.sweep(dac.ch1, np.concatenate((
    np.linspace(0, 25, 10),
    np.linspace(25, 0, 10))), inter_delay=0.1)
# Watch the parameters for up to 100 seconds.
s.watch(max_duration=100, inter_delay=0.5)
# Sweep along two axes, slow then fast.
s.megasweep(
    dac.ch1, np.linspace(0, 10, 10),
    dac.ch2, np.linspace(0, 10, 10), fast_delay=1)

## Where is the data?

In [ ]:
s.dataset